# Snakemake GATK pipeline


In [1]:
! ls

binder	       data		    genotyped	 qc	      schemas
called	       dedup		    howto	 README.md    scripts
config.yaml    envs		    index.ipynb  recal	      Snakefile
dag-rules.svg  filtered		    logs	 rules	      trimmed
dag.svg        gatk-pipeline-stats  mapped	 samples.tsv  units.tsv


In [2]:
! cat Snakefile

include: "rules/common.smk"

##### Target rules #####

rule all:
    input:
        "annotated/all.vcf.gz",
        "qc/multiqc.html",
        "plots/depths.svg",
        "plots/allele-freqs.svg"


##### Modules #####

include: "rules/mapping.smk"
include: "rules/calling.smk"
include: "rules/filtering.smk"
include: "rules/stats.smk"
include: "rules/qc.smk"
include: "rules/annotation.smk"


In [3]:
%%bash

snakemake -np --use-conda

Building DAG of jobs...
Error: Directory cannot be locked. This usually means that another Snakemake instance is running on this directory. Another possibility is that a previous run exited unexpectedly.
Conda environment https:/bitbucket.org/snakemake/snakemake-wrappers/raw/0.27.1/bio/snpeff/environment.yaml will be created.
Conda environment https:/bitbucket.org/snakemake/snakemake-wrappers/raw/0.33.0/bio/multiqc/environment.yaml will be created.
Conda environment https:/bitbucket.org/snakemake/snakemake-wrappers/raw/0.27.1/bio/fastqc/environment.yaml will be created.
Conda environment envs/rbt.yaml will be created.
Conda environment envs/stats.yaml will be created.
Conda environment https:/bitbucket.org/snakemake/snakemake-wrappers/raw/0.27.1/bio/samtools/stats/environment.yaml will be created.
Job counts:
	count	jobs
	1	all
	3	fastqc
	1	hard_filter_calls
	1	merge_calls
	1	multiqc
	1	plot_stats
	3	samtools_stats
	1	select_calls
	1	snpeff
	1	vcf_to_tsv
	14

[Mon Apr 29 08:01:05 2019]

/srv/conda/lib/python3.6/site-packages/snakemake/workflow.py:12: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from collections import OrderedDict
/srv/conda/lib/python3.6/site-packages/snakemake/workflow.py:15: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from operator import attrgetter
/srv/conda/lib/python3.6/site-packages/snakemake/workflow.py:21: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from snakemake.exceptions import RuleException, CreateRuleException, \
